In [ ]:
# default_exp callback

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Callback

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
from pytorch_inferno.utils import to_np

from typing import Optional, Callable
from abc import ABC
from fastcore.all import store_attr
import math
import numpy as np

from torch import Tensor

In [ ]:
# export
class AbsCallback(ABC):
    def __init__(self): pass
    def set_wrapper(self, wrapper) -> None: self.wrapper = wrapper  
    def on_train_begin(self) -> None: pass
    def on_train_end(self) -> None: pass 
    
    def on_epoch_begin(self) -> None: pass
    def on_epoch_end(self) -> None:   pass
    
    def on_batch_begin(self) -> None: pass
    def on_batch_end(self) -> None:   pass
    
    def on_forwards_end(self) -> None: pass
    
    def on_backwards_begin(self) -> None: pass
    def on_backwards_end(self) -> None:   pass
        
    def on_pred_begin(self) -> None: pass
    def on_pred_end(self) -> None:   pass

In [ ]:
# export
class LossTracker(AbsCallback):
    def __init__(self): self.reset()
    def reset(self) -> None: self.losses = {'trn':[], 'val':[]}
    def on_train_begin(self) -> None: self.reset()
    def on_epoch_begin(self) -> None: self.loss,self.cnt = 0,0
        
    def on_epoch_end(self) -> None:
        if self.wrapper.state == 'train':
            self.losses['trn'].append(self.loss/self.cnt)
        else:
            self.losses['val'].append(self.loss/self.cnt)
            print('Train:', self.losses['trn'][-1], 'Valid:', self.losses['val'][-1])
            
    def on_forwards_end(self) -> None:
        sz = len(self.wrapper.x)
        self.loss += self.wrapper.loss_val.data.item()*sz
        self.cnt += sz

In [ ]:
# export
class EarlyStopping(AbsCallback):
    def __init__(self, patience:int):
        store_attr()
        self.reset()
        
    def reset(self) -> None: self.epochs,self.min_loss = 0,math.inf
    def on_train_begin(self) -> None: self.reset()
    def on_epoch_begin(self) -> None: self.loss,self.cnt = 0,0
        
    def on_forwards_end(self) -> None:
        if self.wrapper.state == 'valid':
            sz = len(self.wrapper.x)
            self.loss += self.wrapper.loss_val.data.item()*sz
            self.cnt += sz
        
    def on_epoch_end(self) -> None:
        if self.wrapper.state == 'valid':
            loss = self.loss/self.cnt
            if loss < self.min_loss:
                self.min_loss = loss
                self.epochs = 0
            else:
                self.epochs += 1
            if self.epochs >= self.patience:
                print('Early stopping')
                self.wrapper.stop = True

In [ ]:
# export
class PredHandler(AbsCallback):
    def __init__(self):
        store_attr()
        self.reset()
        
    def reset(self) -> None: self.preds = []
    def on_pred_begin(self) -> None: self.reset()
    def on_pred_end(self) -> None: self.preds = np.vstack(self.preds)
        
    def on_forwards_end(self) -> None:
        if self.wrapper.state == 'test': self.preds.append(to_np(self.wrapper.y_pred))
            
        
    def get_preds(self) -> np.ndarray: return self.preds